<a href="https://www.kaggle.com/code/avd1729/shakspearean-text-generation?scriptVersionId=146556861" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Setting up the environment**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
print(shakespeare_text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [6]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [7]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)  

In [8]:
n_tokens

39

In [9]:
dataset_size

1115394

In [10]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [11]:
list(to_dataset(text_vec_layer(["To be"])[0], length=4))

[(<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 4,  5,  2, 23]])>,
  <tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 5,  2, 23,  3]])>)]

In [12]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

# **Char-RNN Model**

In [13]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                    callbacks=[model_ckpt])

Epoch 1/10
31247/31247 [==============================] - 436s 13ms/step - loss: 1.3995 - accuracy: 0.5716 - val_loss: 1.5931 - val_accuracy: 0.5368
Epoch 2/10
31247/31247 [==============================] - 405s 12ms/step - loss: 1.2963 - accuracy: 0.5963 - val_loss: 1.5769 - val_accuracy: 0.5422
Epoch 3/10
31247/31247 [==============================] - 411s 13ms/step - loss: 1.2774 - accuracy: 0.6004 - val_loss: 1.5644 - val_accuracy: 0.5433
Epoch 4/10
31247/31247 [==============================] - 404s 12ms/step - loss: 1.2666 - accuracy: 0.6030 - val_loss: 1.5510 - val_accuracy: 0.5468
Epoch 5/10
31247/31247 [==============================] - 407s 12ms/step - loss: 1.2601 - accuracy: 0.6042 - val_loss: 1.5442 - val_accuracy: 0.5469
Epoch 6/10
31247/31247 [==============================] - 401s 12ms/step - loss: 1.2546 - accuracy: 0.6058 - val_loss: 1.5434 - val_accuracy: 0.5514
Epoch 7/10
31247/31247 [==============================] - 404s 12ms/step - loss: 1.2505 - accuracy: 0.6066

In [14]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

In [15]:
from pathlib import Path

url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file("shakespeare_model.tgz", url, extract=True)
model_path = Path(path).with_name("shakespeare_model")

352865/352865 [==============================] - 0s 0us/step


In [16]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 0s 400ms/step


'e'

# **Generating Fake Shakespearean text**

In [17]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 1, 1, 1, 0, 0, 0]])>

In [18]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [19]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [20]:
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 34ms/step
To be or not to be so
will i have some strange of the sea,
and the s


In [21]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 36ms/step
To be or not to beg,
i'll might enough to speak at your honour, by p


In [22]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 34ms/step
To be or not to beevicm-vilv!?$m-?gmjz :3?ljb'va;!td&hi.ur3l'-j!3euq


# **Stateful RNN**

In [23]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],
                                                 length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [24]:
def to_non_overlapping_windows(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    return ds.flat_map(lambda window: window.batch(length + 1))

def to_batched_dataset_for_stateful_rnn(sequence, length, batch_size=32):
    parts = np.array_split(sequence, batch_size)
    datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
    ds = tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows))
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

list(to_batched_dataset_for_stateful_rnn(tf.range(20), length=3, batch_size=2))

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 0,  1,  2],
         [10, 11, 12]], dtype=int32)>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 1,  2,  3],
         [11, 12, 13]], dtype=int32)>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 3,  4,  5],
         [13, 14, 15]], dtype=int32)>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 4,  5,  6],
         [14, 15, 16]], dtype=int32)>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 6,  7,  8],
         [16, 17, 18]], dtype=int32)>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 7,  8,  9],
         [17, 18, 19]], dtype=int32)>)]

In [25]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [26]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [27]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_stateful_shakespeare_model",
    monitor="val_accuracy",
    save_best_only=True)

In [28]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
9999/9999 [==============================] - 88s 9ms/step - loss: 1.8619 - accuracy: 0.4520 - val_loss: 1.7033 - val_accuracy: 0.4926
Epoch 2/10
9999/9999 [==============================] - 88s 9ms/step - loss: 1.5611 - accuracy: 0.5293 - val_loss: 1.6149 - val_accuracy: 0.5164
Epoch 3/10
9999/9999 [==============================] - 87s 9ms/step - loss: 1.4844 - accuracy: 0.5485 - val_loss: 1.5758 - val_accuracy: 0.5276
Epoch 4/10
9999/9999 [==============================] - 85s 8ms/step - loss: 1.4459 - accuracy: 0.5586 - val_loss: 1.5607 - val_accuracy: 0.5307
Epoch 5/10
9999/9999 [==============================] - 85s 9ms/step - loss: 1.4227 - accuracy: 0.5641 - val_loss: 1.5475 - val_accuracy: 0.5360
Epoch 6/10
9999/9999 [==============================] - 86s 9ms/step - loss: 1.4072 - accuracy: 0.5681 - val_loss: 1.5409 - val_accuracy: 0.5365
Epoch 7/10
9999/9999 [==============================] - 88s 9ms/step - loss: 1.3966 - accuracy: 0.5709 - val_loss: 1.5391 - val_ac

# **Stateless RNN**

In [29]:
tf.random.set_seed(42)
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [30]:
stateless_model.build(tf.TensorShape([None, None]))

In [31]:
stateless_model.set_weights(model.get_weights())

In [32]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

In [33]:
tf.random.set_seed(42)

print(extend_text("to be or not to be", temperature=0.01))

1/1 [==============================] - 0s 36ms/step
to be or not to be so
have i not see the world in the world in the w
